In [1]:
#@title 급식권 양도 및 선택 시스템

class MealTicketSystem:
    def __init__(self):
        self.available_tickets = []

    def register_ticket(self, student_id):
        self.available_tickets.append(student_id)
        print(f"학생 {student_id}의 급식권이 등록되었습니다.")

    def claim_ticket(self, student_id):
        if self.available_tickets:
            ticket_owner = self.available_tickets.pop(0)
            print(f"학생 {student_id}이(가) 학생 {ticket_owner}의 급식권을 선택했습니다.")
        else:
            print("현재 등록된 급식권이 없습니다.")

system = MealTicketSystem()

def display_menu():
    print("\n1. 급식권 등록")
    print("2. 급식권 선택")
    print("3. 종료")

def main():
    while True:
        display_menu()
        choice = input("선택: ")

        if choice == '1':
            student_id = input("학번을 입력하세요: ")
            system.register_ticket(student_id)
        elif choice == '2':
            student_id = input("학번을 입력하세요: ")
            system.claim_ticket(student_id)
        elif choice == '3':
            print("프로그램을 종료합니다.")
            break
        else:
            print("잘못된 입력입니다. 다시 시도해주세요.")

# 인터랙티브한 실행을 위해 main 함수를 호출하지 않습니다.
# 아래의 예시를 통해 직접 함수를 호출해 보세요.


In [2]:
# 급식권 등록 예시
system.register_ticket("20210001")

# 급식권 선택 예시
system.claim_ticket("20210002")


학생 20210001의 급식권이 등록되었습니다.
학생 20210002이(가) 학생 20210001의 급식권을 선택했습니다.


In [3]:
!pip install ipywidgets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 19.8 MB/s eta 0:00:00


In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import threading
import time
from datetime import datetime, timedelta

class MealTicketSystem:
    def __init__(self):
        self.available_tickets = []
        self.transfer_records = []

    def register_ticket(self, student_id):
        self.available_tickets.append(student_id)
        return f"학생 {student_id}의 급식권이 등록되었습니다."

    def claim_ticket(self, student_id):
        if self.available_tickets:
            ticket_owner = self.available_tickets.pop(0)
            self.transfer_records.append((ticket_owner, student_id))
            return f"학생 {student_id}이(가) 학생 {ticket_owner}의 급식권을 선택했습니다."
        else:
            return "현재 등록된 급식권이 없습니다."

    def get_registered_tickets(self):
        return self.available_tickets

    def get_transfer_records(self):
        return self.transfer_records

    def reset_system(self):
        self.available_tickets = []
        self.transfer_records = []

# 시스템 초기화
system = MealTicketSystem()

# 위젯 정의
register_student_id_input = widgets.Text(
    value='',
    placeholder='학번을 입력하세요',
    description='학번:',
    disabled=False
)

claim_student_id_input = widgets.Text(
    value='',
    placeholder='학번을 입력하세요',
    description='학번:',
    disabled=False
)

register_button = widgets.Button(
    description='급식권 등록',
    disabled=False,
    button_style='success',
    tooltip='급식권 등록',
    icon='check'
)

claim_button = widgets.Button(
    description='급식권 선택',
    disabled=False,
    button_style='info',
    tooltip='급식권 선택',
    icon='hand-paper-o'
)

output = widgets.Output()
registered_tickets_output = widgets.Output()
transfer_records_output = widgets.Output()

# 버튼 클릭 이벤트 핸들러 정의
def on_register_button_clicked(b):
    with output:
        clear_output()
        student_id = register_student_id_input.value
        if student_id:
            result = system.register_ticket(student_id)
            print(result)
            update_registered_tickets_display()
        else:
            print("학번을 입력하세요.")
        register_student_id_input.value = ''

def on_claim_button_clicked(b):
    with output:
        clear_output()
        student_id = claim_student_id_input.value
        if student_id:
            result = system.claim_ticket(student_id)
            print(result)
            update_registered_tickets_display()
            update_transfer_records_display()
        else:
            print("학번을 입력하세요.")
        claim_student_id_input.value = ''

def update_registered_tickets_display():
    with registered_tickets_output:
        clear_output()
        registered_tickets = system.get_registered_tickets()
        if registered_tickets:
            html_content = "<h4>현재 등록된 급식권 학번 목록:</h4><ul>"
            for ticket in registered_tickets:
                html_content += f"<li>{ticket}</li>"
            html_content += "</ul>"
            display(HTML(html_content))
        else:
            display(HTML("<h4>현재 등록된 급식권이 없습니다.</h4>"))

def update_transfer_records_display():
    with transfer_records_output:
        clear_output()
        transfer_records = system.get_transfer_records()
        if transfer_records:
            html_content = "<h4>급식권 양도 기록:</h4><ul>"
            for record in transfer_records:
                html_content += f"<li>학생 {record[0]} -> 학생 {record[1]}</li>"
            html_content += "</ul>"
            display(HTML(html_content))
        else:
            display(HTML("<h4>현재 양도된 급식권이 없습니다.</h4>"))

def reset_system_daily():
    while True:
        now = datetime.now()
        # 자정까지 남은 시간 계산
        next_reset = (now + timedelta(days=1)).replace(hour=0, minute=0, second=0, microsecond=0)
        sleep_time = (next_reset - now).total_seconds()
        time.sleep(sleep_time)
        # 시스템 리셋
        system.reset_system()
        with output:
            clear_output()
            print("시스템이 리셋되었습니다.")
        update_registered_tickets_display()
        update_transfer_records_display()

# 이벤트 핸들러를 버튼에 연결
register_button.on_click(on_register_button_clicked)
claim_button.on_click(on_claim_button_clicked)

# UI 레이아웃
title_html = HTML("<h2>급식권 양도 및 선택 시스템</h2>")
register_section_title = HTML("<h3>급식권 등록</h3>")
claim_section_title = HTML("<h3>급식권 선택</h3>")
registered_tickets_section_title = HTML("<h3>등록된 급식권 목록</h3>")
transfer_records_section_title = HTML("<h3>급식권 양도 기록</h3>")

# UI 표시
display(title_html)
display(register_section_title, register_student_id_input, register_button, output)
display(claim_section_title, claim_student_id_input, claim_button)
display(registered_tickets_section_title, registered_tickets_output)
display(transfer_records_section_title, transfer_records_output)

# 초기 등록된 학번 목록 및 양도 기록 표시
update_registered_tickets_display()
update_transfer_records_display()

# 자정에 시스템을 리셋하는 스레드 시작
reset_thread = threading.Thread(target=reset_system_daily)
reset_thread.daemon = True
reset_thread.start()


Text(value='', description='학번:', placeholder='학번을 입력하세요')

Button(button_style='success', description='급식권 등록', icon='check', style=ButtonStyle(), tooltip='급식권 등록')

Output()

Text(value='', description='학번:', placeholder='학번을 입력하세요')

Button(button_style='info', description='급식권 선택', icon='hand-paper-o', style=ButtonStyle(), tooltip='급식권 선택')

Output()

Output()

In [8]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
with open('/content/drive/My Drive/filename.py', 'w') as f:
    f.write('코드 내용')
